# Setting

In [1]:
import sys

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import AD_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.
cpu is available.


# 1. set param from Front End

In [2]:
beatgan_ad_model_info = {
    'in_c': 51,     # the number of feature
    'hidden_c': 32,
    'latent_c': 16
}

train_parameter = {
    'lr': 0.0001,
    'n_epochs': 1,
    'input_size': 51,
    'batch_size': 32,
    'device': 'cpu',
}

# To be replaced
params1 = {
    'ingestion_param_X': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 
                         'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 
                         'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 
                         'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402','P403', 'P404', 
                         'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504',
                         'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502',
                         'PIT503', 'FIT601', 'P601', 'P602', 'P603']
    },
    'ingestion_param_y': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': ['attack']
    },
    'data_y_flag': 'true',
    'scaler_param':{
        'scaler_flag': 'none',
        'scale_method': 'none',
        'sclaer_path': 'none'
    },
    'transform_param': {
        'split_mode': 'none',
        'data_clean_option': 'none'
    },
    'model_info': {
        'model_purpose': 'anomaly_detection',
        'model_method': 'BeatGAN_ad',
        'model_name': 'None',
        'model_tags': 'tagstest',
        'train_parameter': train_parameter,
        'model_parameter': beatgan_ad_model_info
    }
}

# params1 = {
#     "ingestion_param_X" :{
#         "bucket_name": 'integration',
#         "ms_name" : 'regression_energy_cleanLevel4_trainX',
#         "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
#        'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
#        'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
#     },
#     "ingestion_param_y":{
#         "bucket_name": 'integration',
#         "ms_name" : 'regression_energy_cleanLevel4_trainy',
#         "feature_list":["value"]
#     },
#     'data_y_flag' : 'true',
#     'scaler_param':{
#         'scaler_flag':'scale', #scale_param,
#         'scale_method' :'minmax',
#         'scaler_path' :'./scaler/'
#     },
#     "transform_param":{
#         'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
#         #step_split일 경우만 past_step과 future_step이 존재
#         'data_clean_option' : "false"
#     },
    
#     "model_info" :{
#         'model_purpose' : 'regression',
#         'model_method' : 'GRU_rg',   # 'GRU_rg', 'LSTM_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg'
#         'model_name' : 'None',
#         'model_tags' : 'tagstest',
#         'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
#         'model_parameter' : GRU_rg_model_info
#     }
# }


# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params1

# parameter tunning
params = AD_api.convert_param_for_backend(params)

In [4]:
# for data preparation
def pre_test_df(test_df):
    test_df['attack'] = test_df['attack'].apply(lambda x : 1 if x > 0 else 0)
    split_index = int(len(test_df) * 0.5)
    
    valid_x = test_df.iloc[:split_index,:].drop(columns=['timestamp','attack'])
    valid_y = test_df.iloc[:split_index,:]['attack']

    test_x = test_df.iloc[split_index:,:].drop(columns=['timestamp','attack'])
    test_y = test_df.iloc[split_index:,:]['attack']
    
    return valid_x, valid_y, test_x, test_y

In [5]:
# 2. Data Preparation
# train_X_array, train_y_array, val_X_array, val_y_array = AD_api.train_data_preparation(params, influxdb_client)

# 추후 ML_api 의 데이터 준비 형식으로 대체 가능
import pandas as pd

# from train.csv
df = pd.read_csv('./data/train_ver2.csv')
train_X, train_y = df.drop(columns=['timestamp', 'attack']), df['attack']

# from test.csv
# 보통 train set 에서 train/val 나누지 않는지?
test_df = pd.read_csv('./data/test_ver2.csv')
val_X, val_y, test_X, test_y = pre_test_df(test_df)

In [6]:
print(type(train_X))
print(train_X.shape)  # TBD: [seq_len, n_features]

<class 'pandas.core.frame.DataFrame'>
(49680, 51)


In [7]:
# 3. Training
params = AD_api.AD_train(params, train_X, train_y, val_X, val_y)

KeyError: 'data_param_X'

In [8]:
print(params)

{'data_param_X': {'name': 'SWaT', 'train_path': './data/train_ver2.csv', 'test_path': './data/test_ver2.csv'}, 'model_info': {'model_purpose': 'anomaly_detection', 'model_method': 'beatgan_ad', 'model_name': 'None', 'model_tags': 'tagstest', 'train_parameter': {'lr': 0.0001, 'n_epochs': 5, 'input_size': 51, 'batch_size': 32, 'device': 'cpu'}, 'model_parameter': {'in_c': 51, 'hidden_c': 32, 'latent_c': 16}, 'model_file_path': {'modelFile': {'fileName': 'model.pth', 'filePath': './Models/beatgan_ad/None/SWaT/model.pkl'}}}}


In [ ]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "./meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, params)